In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
!pip install fastai

In [3]:
from fastcore.all import *
from fastai.vision.all import *

In [4]:
PATH = Path(r"C:\Users\Pranav\OneDrive\Desktop\Dog\train")
print(get_image_files(PATH))

[Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/001.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/002.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/003.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/004.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/005.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/006.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/007.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/008.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/009.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/010.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/011.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/012.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/013.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/014.jpg'), Path('C:/Users/Pranav/OneDrive/Desktop/Dog/train/Afghan/015.j

In [5]:
def get_my_files(x):
    return get_image_files(str(PATH)+'\train') + get_image_files(str(PATH)+'\valid') 

In [6]:
def FileSplitter():
    "Split `items` depending on the value of `mask`."
    def _func(pth_in):
        in_valid = False
        if 'valid' in str(pth_in):
            in_valid = True
        return in_valid
    def _inner(o, **kwargs): return FuncSplitter(_func)(o)
    return _inner

In [7]:
dls = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=40),
    get_y = parent_label,
    item_tfms = [Resize(192, method="squish")]
).dataloaders(PATH, bs=32)

In [8]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

C:\Users\Pranav\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Pranav\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,error_rate,time
0,1.532017,0.599873,0.187539,06:23


epoch,train_loss,valid_loss,error_rate,time
0,0.730576,0.527680,0.142228,05:32
1,0.410882,0.420309,0.118313,07:00
2,0.175308,0.389717,0.110132,08:46


In [9]:
learn.summary()

Sequential (Input shape: 32 x 3 x 192 x 192)
Layer (type)         Output Shape         Param #    Trainable 
                     32 x 64 x 96 x 96   
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     32 x 64 x 48 x 48   
MaxPool2d                                                      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                      

In [10]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
import numpy as np
import datetime
import requests
from firebase_admin import db
import json

In [40]:
#cred = credentials.Certificate(r"C:\Users\Pranav\OneDrive\Desktop\cred.json")
#firebase_admin.initialize_app(cred, {'storage': 'pet-tracker-a136c.appspot.com/'})

In [39]:
#cred2 = credentials.Certificate(r"C:\Users\Pranav\OneDrive\Desktop\cred.json")
#firebase_admin.initialize_app(cred2, {'databaseURL': "https://pet-tracker-a136c-default-rtdb.firebaseio.com/Found"})

In [55]:
ref = db.reference('/url')
dic=ref.get()
url=dic['imgurl']
print(url)

https://firebasestorage.googleapis.com/v0/b/pet-tracker-a136c.appspot.com/o/Fri%20Jun%2016%202023%2007%3A37%3A36%20GMT%2B0530%20(India%20Standard%20Time)-gs.jpg?alt=media&token=02ca486a-8325-4836-b017-f9271dff03c7


In [56]:
firebase_admin.get_app()

In [57]:
response = requests.get(url)
with open("image.jpg", "wb") as f:
    f.write(response.content)


In [58]:
img = PILImage.create(r"image.jpg")
breed,_,probs = learn.predict(img)
data={"Dog":breed}
json_object = json.dumps(data, indent=4)
print(breed)

German Sheperd


In [61]:
ref = db.reference('/Found/Dog')
ref.set(breed)